In [181]:
import requests as req
import json
from time import sleep
import pandas as pd

The following code below creates a list of unique id numbers for each NBA team in sports radar database.


In [166]:
rankings=req.get('https://api.sportradar.us/nba/trial/v5/en/seasons/2013/REG/rankings.json?api_key=r9ac8v59fn7uxfeahz9udw7j').json()
sub_info=[]
for i in rankings:
    if i=='conferences':
        for e in rankings[i]:
            for gen_info in e['divisions']:
                for team_info in gen_info['teams']:
                    sub_info.append(team_info['id'])
                   


In [167]:
sub_info
###teams mapped with their team_id for easy access to team data from the sports radar database

['583ecda6-fb46-11e1-82cb-f4ce4684ea4c',
 '583ec9d6-fb46-11e1-82cb-f4ce4684ea4c',
 '583ec70e-fb46-11e1-82cb-f4ce4684ea4c',
 '583eccfa-fb46-11e1-82cb-f4ce4684ea4c',
 '583ec87d-fb46-11e1-82cb-f4ce4684ea4c',
 '583ec7cd-fb46-11e1-82cb-f4ce4684ea4c',
 '583ec5fd-fb46-11e1-82cb-f4ce4684ea4c',
 '583ec773-fb46-11e1-82cb-f4ce4684ea4c',
 '583ec928-fb46-11e1-82cb-f4ce4684ea4c',
 '583ecefd-fb46-11e1-82cb-f4ce4684ea4c',
 '583ecea6-fb46-11e1-82cb-f4ce4684ea4c',
 '583ec8d4-fb46-11e1-82cb-f4ce4684ea4c',
 '583ec97e-fb46-11e1-82cb-f4ce4684ea4c',
 '583ecb8f-fb46-11e1-82cb-f4ce4684ea4c',
 '583ed157-fb46-11e1-82cb-f4ce4684ea4c',
 '583ecfff-fb46-11e1-82cb-f4ce4684ea4c',
 '583ed056-fb46-11e1-82cb-f4ce4684ea4c',
 '583eca2f-fb46-11e1-82cb-f4ce4684ea4c',
 '583ed102-fb46-11e1-82cb-f4ce4684ea4c',
 '583ece50-fb46-11e1-82cb-f4ce4684ea4c',
 '583ecdfb-fb46-11e1-82cb-f4ce4684ea4c',
 '583ec825-fb46-11e1-82cb-f4ce4684ea4c',
 '583ecfa8-fb46-11e1-82cb-f4ce4684ea4c',
 '583ed0ac-fb46-11e1-82cb-f4ce4684ea4c',
 '583ecae2-fb46-

Now, to get every players averages by constructing a pipeline.

In [195]:
##function to get player averages for each team by year and storing the information in a dictionary for further analysis
def get_player_avgs():
    player_avgs={}
    for year in range(2013,2019):
        year=str(year)
        player_avgs[year]={}
        for i_d in sub_info:
            player_info={}
            team_info=req.get('https://api.sportradar.us/nba/{}/{}/{}/seasons/{}/{}/teams/{}/statistics.{}?api_key={}'.format('trial', 'v5', 'en', year, 'REG', i_d,'json', 'r9ac8v59fn7uxfeahz9udw7j')).json()
            sleep(1) ##the program is paused here to void making more calls to the sports radar API than i'm allowed per second
            for record in team_info:
                if record=='players':
                    for data in team_info[record]:
                        player_info[data['full_name']]=data['average']
                    player_avgs[year][team_info['name']]=player_info
    return player_avgs
    

In [196]:
records=get_player_avgs()

In [197]:
df=pd.DataFrame(records)

In [198]:
len(records)##checking that the acccurate number of seasons have been loaded

6

In [202]:
len(records['2018']) ##checking that the accurate number of teams are loaded. 

30

In [204]:
df.head()

,2013,2014,2015,2016,2017,2018
76ers,"{'Casper Ware': {'minutes': 12.9, 'points': 5....","{'Ish Smith': {'minutes': 27.1, 'points': 12.0...","{'Ish Smith': {'minutes': 32.4, 'points': 14.7...","{'Dario Saric': {'minutes': 26.3, 'points': 12...","{'Dario Saric': {'minutes': 29.6, 'points': 14...","{'Dario Saric': {'minutes': 30.5, 'points': 11..."
Bucks,"{'O.J. Mayo': {'minutes': 25.9, 'points': 11.7...","{'Jared Dudley': {'minutes': 23.8, 'points': 7...","{'O.J. Mayo': {'minutes': 26.6, 'points': 7.8,...","{'Mirza Teletovic': {'minutes': 16.2, 'points'...","{'Mirza Teletovic': {'minutes': 15.9, 'points'...","{'Tim Frazier': {'minutes': 14.8, 'points': 4...."
Bulls,"{'Jimmy Butler': {'minutes': 38.7, 'points': 1...","{'Jimmy Butler': {'minutes': 38.7, 'points': 2...","{'Justin Holiday': {'minutes': 18.9, 'points':...","{'Denzel Valentine': {'minutes': 17.1, 'points...","{'Denzel Valentine': {'minutes': 27.2, 'points...","{'Denzel Valentine': {'minutes': 0, 'points': ..."
Cavaliers,"{'Carrick Felix': {'minutes': 5.4, 'points': 2...","{'A.J. Price': {'minutes': 7.9, 'points': 2.0,...","{'Sasha Kaun': {'minutes': 3.8, 'points': 0.9,...","{'Deron Williams': {'minutes': 20.3, 'points':...","{'Jordan Clarkson': {'minutes': 22.6, 'points'...","{'Matthew Dellavedova': {'minutes': 19.9, 'poi..."
Celtics,"{'Courtney Lee': {'minutes': 16.8, 'points': 7...","{'Marcus Thornton': {'minutes': 16.4, 'points'...","{'John Holland': {'minutes': 0, 'points': 0, '...","{'James Young': {'minutes': 7.6, 'points': 2.3...","{'Xavier Silas': {'minutes': 3.5, 'points': 0....","{'Guerschon Yabusele': {'minutes': 5.7, 'point..."


In [207]:
df.loc['76ers']['2013']
##testing to see that the series contains the right information. 

{'Adonis Thomas': {'assists': 0.5,
  'blocked_att': 0.0,
  'blocks': 0.0,
  'def_rebounds': 0.0,
  'efficiency': 2.5,
  'fast_break_att': 0,
  'fast_break_made': 0,
  'fast_break_pts': 0,
  'field_goals_att': 2.5,
  'field_goals_made': 1.5,
  'flagrant_fouls': 0.0,
  'fouls_drawn': 0.0,
  'free_throws_att': 0.0,
  'free_throws_made': 0.0,
  'minutes': 6.5,
  'off_rebounds': 0.0,
  'offensive_fouls': 0.0,
  'personal_fouls': 2.0,
  'points': 3.5,
  'points_in_paint': 0,
  'points_in_paint_att': 0.0,
  'points_in_paint_made': 0.0,
  'points_off_turnovers': 0,
  'rebounds': 0.0,
  'second_chance_att': 0,
  'second_chance_made': 0,
  'second_chance_pts': 0,
  'steals': 0.0,
  'three_points_att': 1.0,
  'three_points_made': 0.5,
  'true_shooting_att': 2.5,
  'turnovers': 0.5,
  'two_points_att': 1.5,
  'two_points_made': 1.0},
 'Arnett Moultrie': {'assists': 0.2,
  'blocked_att': 0.33,
  'blocks': 0.33,
  'def_rebounds': 2.0,
  'efficiency': 4.58,
  'fast_break_att': 0.0,
  'fast_break_made